# Create smooth elevation contours

![Contours for import to Mapbox](graphics/qa_15.jpg)

In [ ]:
################ DO NOT EDIT ################ 
# 
# Import ArcGIS package
import arcpy
# Subprocess allows us to issue commands on the command line
import subprocess
# Zip utlity to extract files
from zipfile import ZipFile
# zip up a directory
import shutil

In [ ]:
# Directory for your root GIS folder
# Got to use double backslash because of other modules
# 💡💡💡
out_directory = "c:\\YodasGIS\\"

# Project name - creates a folder in project directory
# 💡💡💡
project = "contours"

# Output geodatabase and folder names
# 💡💡💡
out_geodb = "workspace.gdb"
out_shp = "shapefiles"

# Layers created from previous module on lidar and 3D mapping, e.g., natural bridge arch
# 💡💡💡
#  DEM (In geodatabase or as a GeoTIFf)
in_dem = "c:\\YodasGIS\\natbridge\\workspace.gdb\\natbridge_dem_5ft"
# Buffer polygon for area of interest
aoi = "c:\\YodasGIS\\natbridge\\workspace.gdb\\natbridge_2000ft"

# Output parameters
# 💡💡💡
# Contour interval
contour_interval = 10
# Index contour
contour_index = 100


contour_name = f"contour_{contour_interval}ft"
print("Finished")

In [ ]:
################ DO NOT EDIT ################ 
# 
# Create folders
subprocess.run(f'mkdir {out_directory}{project}', shell=True)
subprocess.run(f'mkdir {out_directory}{project}\\{out_shp}', shell=True)
completed = subprocess.run(f'dir {out_directory}', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))

print("Finished")

In [ ]:
################ DO NOT EDIT ################ 
# 
arcpy.env.overwriteOutput = True

In [ ]:
################ DO NOT EDIT ################ 
# 
# Create project geodatabase
arcpy.CreateFileGDB_management(f'{out_directory}{project}', out_geodb)

print("Finished")

In [ ]:
################ DO NOT EDIT ################ 
# 
# Create project default geodatabase
out_path = f'{out_directory}{project}\\{out_geodb}'
arcpy.env.workspace = out_path
completed = subprocess.run(f'dir {out_directory}{project}', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))

print("Finished")

In [ ]:
################ DO NOT EDIT ################ 
# 
neighborhood = arcpy.sa.NbrCircle(3,'CELL')
outFocalStat = arcpy.sa.FocalStatistics(in_dem, neighborhood, "MEAN")
outFocalStat.save(f"{out_path}\\avg_elev_30ft")
arcpy.Contour_3d (f"{out_path}\\avg_elev_30ft", f"{out_path}\\{contour_name}", contour_interval)

print("Finished")

In [ ]:
################ DO NOT EDIT ################ 
# 
# Find index contours and label them in a new field

y = contour_index

code_block = """
def is_index_contour(x, y):
    if x % y == 0:
        return 1
    else:
        return 0
"""     
arcpy.AddField_management (f"{out_path}\\{contour_name}", "index", "SHORT")        
arcpy.CalculateField_management(f"{out_path}\\{contour_name}", "index", f"is_index_contour(!CONTOUR!, {y})", "PYTHON3", code_block)

print("Finished")

In [ ]:
################ DO NOT EDIT ################ 
# 
# Clip contours to area of interest
arcpy.Clip_analysis (f"{out_path}\\{contour_name}", aoi, f'{out_directory}{project}\\{out_shp}\\{contour_name}_aoi.shp')

print("Finished")

In [ ]:
################ DO NOT EDIT ################ 
# 
# Create archive of ShapeFile for upload to Mapbox
shutil.make_archive(f"{out_directory}{project}\\{contour_name}_aoi", 'zip', f'{out_directory}{project}\\{out_shp}\\')

print("Finished")